<h3>0. Exploratory analysis</h3>

In [1]:
# tools
import pickle

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [29]:
df_training.head()

,answer_paragraph,docid,question,text,NE_question,NE_text,NE_paragraph,answer_type,keywords_question,question_type,POS_questions,answer_found,corrected_answer,NE_corrected_answer,NE_answer_found,label
0,23,0,A kilogram could be definined as having a Plan...,6966662606895999999♠6.62606896×10−34 j⋅s,[],"[(6966662606895999999 ♠ 6.62606896, NUMBER), (...","[[(general, TITLE), (2011, DATE)], [], [(one, ...",NUMBER,"[kilogram, definined, planck, constant, value]",what,"[NN, VBN, NNP, NN, NN]","Possible new definitions include ""the mass of ...",6966662606895999999♠6.62606896×10−34 J⋅s,"[(6966662606895999999 ♠ 6.62606896, NUMBER), (...","[(7050135639273999999 ♠ 135639274 ×, NUMBER), ...",NUMBER
1,22,0,What is the shape of the object that establish...,cylinder,[],[],"[[], [], [(1889, DATE), (paris, CITY)], [(1889...",None,"[shape, object, establish, base, unit, kilogram]",what,"[NN, NN, VBZ, JJ, NN, NN]",The most urgent unit on the list for redefinit...,cylinder,[],"[(1889, DATE), (paris, CITY)]",OTHER
2,12,0,What example is given as another paired relati...,time vs. energy,[],[],"[[], [], [(one, NUMBER)], [(fourier, LOCATION)]]",None,"[example, give, pair, relationship, relate, st...",what,"[NN, VBN, JJ, NN, VBN, JJ, NN]",One example is time vs. energy.,time vs. energy,[],"[(one, NUMBER)]",OTHER
3,1,0,What does the Planck Constant refer to?,quantum of action,[],[],"[[], [(planck, PERSON)], [(now, DATE)], [], []]",None,"[doe, planck, constant, refer]",what,"[VBZ, NNP, NNP, NN]","Instead, it must be some multiple of a very sm...",quantum of action,[],"[(now, DATE)]",OTHER
4,10,0,When was the first quantized model of the atom...,1913,"[(first, ORDINAL), (model, TITLE)]","[(1913, DATE)]","[[(niels bohr, PERSON), (first, ORDINAL), (mod...",DATE,"[wa, first, quantize, model, atom, introduce]",when,"[VBD, JJ, JJ, NN, NN, VBD]",Niels Bohr introduced the first quantized mode...,1913,"[(1913, DATE)]","[(niels bohr, PERSON), (first, ORDINAL), (mode...",TIME


In [100]:
import json
import pandas as pd

df_devel=pd.read_pickle('project_files/df_devel.pkl')
df_docs=pd.read_json('project_files/documents.json')
df_testing=pd.read_pickle('project_files/df_testing.pkl')
df_training=pd.read_pickle('project_files/df_training.pkl')
question_learning_dataset = df_training[df_training.answer_type.notnull()]
question_devel_dataset = df_devel[df_devel.answer_type.notnull()]
NER_corpus=load_obj('ner_corpus')
model = load_obj('random_forest')

In [298]:
df_devel=pd.read_pickle('project_files/df_devel.pkl')
question_devel_dataset = df_devel[df_devel.answer_type.notnull()]

In [3]:
print(len(question_learning_dataset),len(question_devel_dataset))

29174 1969


In [10]:
from nltk.tokenize.punkt import PunktSentenceTokenizer,PunktTrainer

tokenizer = load_obj('punk_tokenizer')
tokenizer._params.abbrev_types.add('ii')
tokenizer._params.abbrev_types.add('dr')

questionwords = set(["who", "what", "where", "when", "why", "how", "whose", "which", "whom","whats","what's","whos"])
passiveQuestions = set(["can", "could", "would", 
                   "was", "were","am","is", "are", "will","shall",
                   "did","do","does",
                   "had", "have","has",
                   "as","that","in",
                   "give an example","name"])


1. Find Keywords
2. Answer types - Using answer type taxonomy
3. Query formulation -> Keywords
4. Go to each document and check the frequency distribution of words and pick the document if one of the query words are present in document. Create a rank with that score
5. Find the paragraphs -> Discard irrelevant paragraphs. Use NE,Keywords, longest exact keywords. Put same weight for now and calculate the score of paragraphs. Rank each of the paragraphs in the document. We have to use the original answer and match the answer type
6. Find candidate answers -> Use supervised ML method
7. Merge candidate answers -> Use NER
8. Pick the best answer -> Logistic regression

<h3>1. Question processing</h3>

Configuring Stanford CoreNLP . Link -> https://blog.manash.me/configuring-stanford-parser-and-stanford-ner-tagger-with-nltk-in-python-on-windows-f685483c374a

In [11]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tag.stanford import CoreNLPNERTagger
from itertools import groupby

stopwords = set(nltk.corpus.stopwords.words('english')) 


def get_Name_Entity_NLTK(data):
    results=[]
    for sentence in data:
        ne_chunked_sents = ne_chunk(pos_tag(word_tokenize(sentence)))
        result = []

        for tagged_tree in ne_chunked_sents:

            if hasattr(tagged_tree, 'label'):
                entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
                entity_type = tagged_tree.label() # get NE category
                result.append((entity_name, entity_type))
        results.append(result)

    return results

def get_Name_Entity_Sentence(sentence):
    st = CoreNLPNERTagger(url='http://localhost:9000')
    tokenized_text = nltk.word_tokenize(sentence)
    classified_text = st.tag(tokenized_text)
    result = []
    
    for tag, chunk in groupby(classified_text, lambda x:x[1]):
       if tag != "O":
            word = " ".join(w for w, t in chunk)
            result.append((word.lower(), tag))
    
    return result


def addNameEntity(df,feature,func):
    if 'NE'+"_"+feature in df:
        df = df.drop('NE'+"_"+feature, axis=1)
    df["NE"+"_"+feature] = func(df[feature])
    
    return df

In [206]:
def get_question_type(question):
    found  = False
    result = 'other'
    question_tokens = nltk.word_tokenize(question)
    for token in question_tokens:
        if token in questionwords:
            found = True
            result = token
    if not found:
        for token in question_tokens:
            if token in passiveQuestions:
                found = True
                result = token

    return result

Get Keywords

In [12]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

POS = set(["NN","NNS","NNP","NNPS","CD","JJ","VB","VBD","VBG","VBN","VBP","VBZ"]) 

stopwords = set(nltk.corpus.stopwords.words('english')) 


def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,wn.NOUN)
    if (lemma == word):
        lemma = lemmatizer.lemmatize(word,wn.VERB)
        
    return lemma

def get_keyword(data):
    result = []
    sentence=data
    tokenized_text = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokenized_text)
    for text,pos in tagged:
        text = lemmatize(text.lower())
        if text not in stopwords:
            if pos in POS:
                result.append(text)
                
    return result

def get_keyword_paragraph(data):
    results=[]
    tokenized_sentence = tokenizer.tokenize(data)
    for sentence in tokenized_sentence:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def get_keyword_all(data):
    results=[]
    for sentence in data:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def add_keywords(df,feature):
    if 'keywords'+"_"+feature in df:
        df = df.drop('keywords'+"_"+feature, axis=1)
    df['keywords'+"_"+feature]=get_keyword_all(df[feature])
    return df

def get_number_of_common_kewyords(question_keywords,answer_sentence_keywords):
    sum_keywords=0
    for qkey in question_keywords:
        if qkey in answer_sentence_keywords:
            sum_keywords+=1
    
    return sum_keywords

<h4>Train a classifier</h4>

In [295]:
# BOW extraction for passages and questions
def get_passages_bow(passages):
    passage_bow={}
    for passage in passages:
        for token in nltk.word_tokenize(passage):
            if token not in stopwords: 
                word=lemmatize(token.lower())
                passage_bow[word] = passage_bow.get(word, 0) +  1
    
    return passage_bow

def get_sentences_bow(sentences):
    sentence_bow={}
    
    for sentence in sentences:
        for token in nltk.word_tokenize(sentence):
            if token not in stopwords:
                word=lemmatize(token.lower())
                sentence_bow[word] = sentence_bow.get(word, 0) +  1
    
    return sentence_bow

def get_question_bow(question):
    question_bow={}
    question_bow[get_question_type(question)]=1
    for token in nltk.word_tokenize(question):
        if token not in stopwords: 
            word=lemmatize(token.lower())
            question_bow[word] = question_bow.get(word, 0) +  1
                
    return question_bow

def get_training_question_bow(question,keywords,qt):
    question_bow={}
    question_bow[qt]=1
    for token in nltk.word_tokenize(question):
        if token not in stopwords: 
            word=lemmatize(token.lower())
            if word in keywords:
                question_bow[word] = question_bow.get(word, 0) +  1
                
    return question_bow

def get_training_question_bow2(question,keywords,qt):
    question_bow={}
    question_bow[qt]=1
    for token in nltk.word_tokenize(question):
        if token not in stopwords: 
            word=lemmatize(token.lower())
            question_bow[word] = question_bow.get(word, 0) +  1
#     for word in keywords:
#         question_bow[word] = question_bow.get(word, 0) +  1
                
    return question_bow

In [292]:
def get_feature_questions(questions, keywords,qt):
    qs = []
    for i,question in enumerate(questions):
        q_bow = get_training_question_bow(question,keywords,qt[i])
        qs.append(q_bow)
        
    return qs

# def get_feature_questions2(questions, keywords,qt):
#     qs = []
#     for i,question in enumerate(questions):
#         q_bow = get_training_question_bow(question,keywords,qt[i])
#         qs.append(q_bow)
        
#     return qs

def get_feature_questions2(questions, keywords,qt):
    qs = []
    for i,question in enumerate(questions):
        q_bow = get_training_question_bow2(question,keywords[i],qt[i])
        qs.append(q_bow)
        
    return qs

def get_feature_question(question, keywords,qt):
#     print(question,keywords,qt)
    q_bow = get_training_question_bow(question,keywords,qt)
      
    return q_bow

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def check_results(predictions, classifications):
    print("Accuracy:")
    print(accuracy_score(classifications,predictions))
    print(classification_report(classifications,predictions))

In [281]:
question_learning_dataset.head()

,answer_paragraph,docid,question,text,NE_question,NE_text,NE_paragraph,answer_type,keywords_question,question_type,POS_questions,answer_found,corrected_answer,NE_corrected_answer,NE_answer_found,label
0,23,0,A kilogram could be definined as having a Plan...,6966662606895999999♠6.62606896×10−34 j⋅s,[],"[(6966662606895999999 ♠ 6.62606896, NUMBER), (...","[[(general, TITLE), (2011, DATE)], [], [(one, ...",NUMBER,"[kilogram, definined, planck, constant, value]",what,"[NN, VBN, NNP, NN, NN]","Possible new definitions include ""the mass of ...",6966662606895999999♠6.62606896×10−34 J⋅s,"[(6966662606895999999 ♠ 6.62606896, NUMBER), (...","[(7050135639273999999 ♠ 135639274 ×, NUMBER), ...",NUMBER
4,10,0,When was the first quantized model of the atom...,1913,"[(first, ORDINAL), (model, TITLE)]","[(1913, DATE)]","[[(niels bohr, PERSON), (first, ORDINAL), (mod...",DATE,"[wa, first, quantize, model, atom, introduce]",when,"[VBD, JJ, JJ, NN, NN, VBD]",Niels Bohr introduced the first quantized mode...,1913,"[(1913, DATE)]","[(niels bohr, PERSON), (first, ORDINAL), (mode...",TIME
5,3,0,What scientist first studied black body radiat...,kirchhoff,"[(scientist, TITLE), (first, ORDINAL)]",[],"[[(the last years of the nineteenth century, D...",PERSON,"[scientist, study, black, body, radiation]",what,"[NN, VBD, JJ, NN, NN]","In the last years of the nineteenth century, P...",Kirchhoff,"[(kirchhoff, PERSON)]","[(the last years of the nineteenth century, DA...",PERSON
6,11,0,Who helped to give the correct quantization ru...,schrödinger,"[(1926, DATE)]","[(schrödinger, PERSON)]","[[(bohr, PERSON), (now, DATE)], [(first, ORDIN...",PERSON,"[help, give, correct, quantization, rule, elec...",who,"[VBD, VB, JJ, NN, NNS, NNS, CD]",The correct quantization rules for electrons –...,Schrödinger,"[(schrödinger, PERSON)]","[(bohr, PERSON), (model, TITLE), (heisenberg, ...",PERSON
9,2,0,How much energy is contained in the light to w...,7005216000000000000♠216 kj/mol,[],"[(7005216000000000000, NUMBER), (216, NUMBER)]","[[], [(555, NUMBER), (7014540000000000000 ♠ 54...",NUMBER,"[much, energy, contain, light, human, eye, sen...",how,"[JJ, NN, VBN, NN, JJ, NNS, JJ]",The result is that green light of wavelength 5...,7005216000000000000♠216 kJ/mol,"[(7005216000000000000, NUMBER), (216, NUMBER)]","[(555, NUMBER), (7005216000000000000 ♠ 216, NU...",NUMBER


In [302]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer


    
# get the most common words from answer sentences (we can twek this for paragraph)
answer_sentences_bow=get_sentences_bow(question_learning_dataset[question_learning_dataset['answer_found'].notnull()]['answer_found'])
answer_keywords = set([word for word, count in answer_sentences_bow.items()])


qs_training=get_feature_questions(list(question_learning_dataset.question),answer_keywords,list(question_learning_dataset.question_type))
qs_dev=get_feature_questions(list(question_devel_dataset.question),answer_keywords,list(question_devel_dataset.question_type))

# qs_training=get_feature_questions2(list(question_learning_dataset.question),list(question_learning_dataset.keywords_question)
#                                    ,list(question_learning_dataset.question_type))
# qs_dev=get_feature_questions2(list(question_devel_dataset.question),list(question_devel_dataset.keywords_question)
#                              ,list(question_devel_dataset.question_type))


In [303]:
from sklearn.ensemble import RandomForestClassifier

if (len(qs_training)>0 and len(list(question_learning_dataset.question_type))>0):
    # fit vectorizer
    vectorizer = DictVectorizer()
    
    X_train_dtm = vectorizer.fit_transform(qs_training)
    X_dev_dtm = vectorizer.transform(qs_dev)
    

    model=RandomForestClassifier(n_estimators = 300, max_depth = 60, criterion = 'entropy')
    
    # tag the answers
    # fit a logistic regression model to the data 
    # build classifier
    #model = MultinomialNB(2, False, None)

    # train the model using X_train_dtm 
    model.fit(X_train_dtm, list(question_learning_dataset.label))
    
    y_predicted_class = model.predict(X_dev_dtm)
    check_results(y_predicted_class,list(question_devel_dataset.label))

Accuracy:
0.7160995429151854


C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

    LOCATION       0.70      0.56      0.62       226
        MISC       0.00      0.00      0.00        51
        NAME       0.79      0.27      0.40       184
      NUMBER       0.89      0.77      0.83       466
ORGANIZATION       1.00      0.03      0.05        72
      PERSON       0.57      0.92      0.70       548
        TIME       0.85      0.87      0.86       422

 avg / total       0.74      0.72      0.69      1969



In [304]:
    y_predicted_class = model.predict(X_train_dtm)
    check_results(y_predicted_class,list(question_learning_dataset.label))

Accuracy:
0.8266607253033523
              precision    recall  f1-score   support

    LOCATION       0.87      0.74      0.80      3851
        MISC       1.00      0.19      0.32       734
        NAME       0.96      0.36      0.53      1877
      NUMBER       0.96      0.89      0.92      6403
ORGANIZATION       1.00      0.29      0.45      1327
      PERSON       0.67      0.97      0.79      8392
        TIME       0.94      0.95      0.95      6590

 avg / total       0.87      0.83      0.81     29174



In [17]:
save_obj(model,'random_forest1')
save_obj(vectorizer,'Vectorizer')
save_obj(qs_training,'qs_training')
save_obj(qs_dev,'qs_dev')

In [305]:
model = load_obj('random_forest')
vectorizer = load_obj('Vectorizer')
print("done")

done


<h3>2. Candidate answering generation</h3>

<h4> Get a score for the passage to filter the most relevant passages</h4>


First, we will set up useful functions to extract term frequencies to build the vector space model

In [23]:
# create a document-term matrix
docs=get_document_term_passsages(df_docs)
#docs

In [24]:
# create a vector space model we need to define a score function
# first I will use tf-idf
doc_term_freqs = {}
for docid,dic_passages in docs.items():
    passage_dic = {}
    for passage_id, terms in dic_passages.items():
        term_freqs = extract_term_freqs(terms)
        passage_dic[passage_id] = term_freqs
    doc_term_freqs[docid] = passage_dic

doc_freqs = compute_doc_freqs(doc_term_freqs)


<b>Improvement:</b> Use BM25

Create an inverted index for query processing. Inverted index will not change from query to query. Here we can improve how the weight is defined for the posting list tuple for each term (docid,weight)

In [18]:
vsm_inverted_index_all=load_obj('vsm_inverted_index_corpus')

Query the VSM creating a score for each document (passage) and returning the top k

In [19]:
from collections import Counter
# get a list of paragraphs ordered by relevance on the question
def query_vsm(query, index):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator

## end copied code

<h3>3. Candidate answering scoring</h3>

In [21]:
import re
def correct_answer_space(predicted,predicted_answer_sentence):
    
    tokens=nltk.word_tokenize(predicted)
    pattern='.*('
    for token in  tokens:
        pattern=pattern+token+'\s*'
    pattern=pattern+').*'

    reg=re.compile(pattern,re.IGNORECASE)
    if len(re.findall(reg,predicted_answer_sentence)):
        predicted=re.findall(reg,predicted_answer_sentence)[0].strip()
    
    return predicted

def correct_answer_pattern(predicted):
    corrected=predicted
    
    # symbol % based pattern
    pattern_percentaje='(.*[0-9])(%.*)'
    reg=re.compile(pattern_percentaje,re.IGNORECASE)
    result=re.findall(reg,predicted)
    if len(result)>0:
        groups=result[0]
        corrected=groups[0]+' '+groups[1]
    
    
    # date pattern
    #pattern_date='(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+(\d{1,2})(,.*)'
    pattern_date='(January|February|March|April|May|June|July|August|September|October|November|December)\s+([0-9]{1,2})(,.*)([0-9]{4})'


    reg=re.compile(pattern_date,re.IGNORECASE)
    result=re.findall(reg,predicted)

    if len(result)>0:
        groups=result[0]   
        corrected=(groups[0]+' '+groups[1]+' '+groups[2]+groups[3])


    return corrected
    
    
def isAnswerInSentence(answer,answer_sentence):
    #print('Eval:',answer)
    #print('In:',answer_sentence)
    tokens=nltk.word_tokenize(answer)
    pattern='.*('
    for token in  tokens:
        pattern=pattern+token+'\s*'
    pattern=pattern+').*'

    
    reg=re.compile(pattern,re.IGNORECASE)
    if len(re.findall(reg,answer_sentence))>0:
        return True
    
    return False




In [ ]:
label={'NUMBER':'NUMBER','DATE':'TIME','PERSON':'PERSON','ORGANIZATION':'ORGANIZATION',
      'MISC':'MISC','MONEY':'NUMBER','COUNTRY':'LOCATION',
      'PERCENT':'NUMBER','TITLE':'PERSON','STATE_OR_PROVINCE':'LOCATION',
      'CAUSE_OF_DEATH':'NAME','DURATION':'TIME','CRIMINAL_CHARGE':'NAME',
       'CITY':'LOCATION','RELIGION':'NAME','SET':'TIME','NATIONALITY':'NAME',
       'IDEOLOGY':'NAME','ORDINAL':'NUMBER','TIME':'TIME','URL':'ORGANIZATION',None:'OTHER','LOCATION':'LOCATION'}

In [275]:
df_devel1.head()

,answer_paragraph,docid,question,text,question_type,NE_question,NE_text,keywords_question,answer_type,POS_questions,answer_found,corrected_answer,NE_corrected_answer,corrected_answer_type,label,NE_answer_found
0,5,380,On what date did the companies that became the...,"june 16 , 1911",what,[],"[(june 16 , 1911, DATE)]","[date, company, become, computing-tabulating-r...",DATE,"[NN, NNS, VBD, NNP, NNP, VB, VBN]","On June 16, 1911, their four companies were co...","June 16, 1911","[(june 16 , 1911, DATE)]",DATE,TIME,"[(june 16 , 1911, DATE), (four, NUMBER), (new ..."
1,22,380,What percentage of its desktop PCs does IBM pl...,5 %,what,"[(ibm, ORGANIZATION)]","[(5 %, PERCENT)]","[percentage, desktop, pc, doe, ibm, plan, inst...",PERCENT,"[NN, JJ, NN, VBZ, NNP, NN, VB, NNP, NNP]",IBM plans to install Open Client on 5% of its ...,5%,"[(5 %, PERCENT)]",PERCENT,NUMBER,"[(ibm, ORGANIZATION), (5 %, PERCENT)]"
2,16,380,What year did IBM hire its first black salesman?,1946,what,"[(year, DURATION), (ibm, ORGANIZATION), (first...","[(1946, DATE)]","[year, ibm, hire, first, black, salesman]",DATE,"[NN, NNP, VB, JJ, JJ, NN]","In 1946, the company hired its first black sal...",1946,"[(1946, DATE)]",DATE,TIME,"[(1946, DATE), (first, ORDINAL), (salesman, TI..."
3,4,380,"IBM made an acquisition in 2009, name it.",spss,in,"[(ibm, ORGANIZATION), (2009, DATE)]",[],"[ibm, make, acquisition, 2009, name]",ORGANIZATION,"[NNP, VBD, NN, CD, NN]",IBM acquired Kenexa (2012) and SPSS (2009) and...,SPSS,"[(spss, ORGANIZATION)]",ORGANIZATION,ORGANIZATION,"[(ibm, ORGANIZATION), (kenexa, PERSON), (2012,..."
4,2,380,This IBM invention is known by the acronym UPC...,universal product code,what,"[(ibm, ORGANIZATION), (upc, ORGANIZATION)]",[],"[ibm, invention, know, acronym, upc, full, name]",None,"[NNP, NN, VBN, NN, NNP, JJ, NN]",Notable company inventions or developments inc...,Universal Product Code,[],None,OTHER,"[(teller, TITLE), (upc, ORGANIZATION), (fortra..."


In [537]:
df_result_devel=pd.DataFrame(columns=['id','question','paragraph','retrieved paras','predicted_paragraph',
                                      'paragraph_found','sentence','predicted_sentence','predicted_sentence_id',
                                      'sentence_found','answer','predicted_answer','NE_question','label'])
df_devel1=df_devel.iloc[0:100]
count = 0
for index, row in df_devel.iterrows():
    question=row['question']
    docid=row['docid']
    ida=index
    question_keywords=get_keyword(question)
    results = query_vsm(question_keywords, vsm_inverted_index_all[docid])
    documents_ranked=results.most_common(2)
#     print(documents_ranked)
    par_retrieved = False
    possible_par=[par[0] for par in documents_ranked]
    if(row['answer_paragraph'] in set(possible_par)):
        par_retrieved = True
        count  = count + 1
    df_result_devel.loc[ida] = [docid,question,row['answer_paragraph'],
                                possible_par,possible_par,
                                par_retrieved,row['answer_found'],None,None,False,row['text'],None,
                                row['NE_question'],row['label']]

print(count*1.0/len(df_devel))

0.8120762027768809


In [538]:
df_result_devel = df_result_devel[df_result_devel['paragraph_found']==True]
print(len(df_result_devel))

2515


In [539]:
df_best_sentences=pd.DataFrame(columns=['doc_id','para_id','original','sentence_id','sentence_text','score','sentence'])
count  = 0
for index,row in df_result_devel.iterrows():
    NE_question = row['NE_question']
    question = row['question']
    answer = row['sentence']
    docid = row['id']
    candidate_answers = {}
    possible_paragraphs = row['predicted_paragraph']
    print(possible_paragraphs)
    sentences_check = Counter()
    for paragraph_id in possible_paragraphs:
        paragraph = df_docs.iloc[row['id']]['text'][paragraph_id]
        candidate_answers[paragraph_id] = tokenizer.tokenize(paragraph)
        question_keywords=get_keyword(question)
        for sentence_index,sentence in enumerate(candidate_answers[paragraph_id]):
            NER_sentence=NER_corpus[docid][paragraph_id][sentence_index]
            answer_sentence_keywords = get_keyword(sentence)
            common_keywords=get_number_of_common_kewyords(question_keywords,answer_sentence_keywords)
            common_entities = get_number_of_common_entities(NER_sentence,NE_question)

            # longest exact sequence of keywords
            longest_exact_sequence=0
            for i in range(len(question_keywords)):
                if i < len(answer_sentence_keywords):
                    if question_keywords[i] in answer_sentence_keywords[i]:
                        longest_exact_sequence+=1

            # proximity
            proximity=0
            question_keywords_span=question_keywords.copy()
            index_qk=0
            while len(question_keywords_span)>0 and index_qk<len(question_keywords_span):
                proximity+=1
                if question_keywords_span[index_qk] in answer_sentence_keywords:
                    question_keywords_span.pop(index_qk)
                index_qk+=1

            # n-gram overlap
            bigrams_question =  nltk.bigrams([lemmatize(token) for token in nltk.word_tokenize(question)])
            ngram_overlap=0

            for bigram_question in bigrams_question:
                bigrams_sentence = nltk.bigrams([lemmatize(token) for token in nltk.word_tokenize(sentence)])
                for bigram_sentence in bigrams_sentence:
                    if bigram_question == bigram_sentence:
                        ngram_overlap+=1
            
            sentence_score = common_keywords + common_entities + longest_exact_sequence - proximity + ngram_overlap
            df_best_sentences.loc[len(df_best_sentences)]=[row['id'],
                                                       paragraph_id,row['paragraph'],sentence_index,
                                                       sentence,sentence_score,row['sentence']]
            sentences_check[(paragraph_id,sentence_index)] = sentence_score
    
    best = sentences_check.most_common()
#     print(best[0][0][0],best[0][0][1])
    df_result_devel.at[index, 'predicted_sentence_id'] = best[0][0][1]
    df_result_devel.at[index, 'predicted_sentence'] = candidate_answers[best[0][0][0]][best[0][0][1]]
    print(index)
    if(row['sentence']==candidate_answers[best[0][0][0]][best[0][0][1]]):
        count = count + 1
        df_result_devel.at[index, 'sentence_found'] = True
print(count*1.0/len(df_result_devel))
print("Done")

[5, 0]
0
[22, 16]
1
[16, 17]
2
[2, 17]
4
[1, 13]
5
[25, 14]
6
[20, 19]
8
[22, 20]
9
[23, 17]
10
[20, 22]
11
[18, 17]
12
[8, 0]
13
[25, 0]
14
[17, 18]
15
[16, 15]
16
[10, 23]
17
[11, 22]
18
[10, 4]
20
[24, 2]
21
[21, 8]
22
[3, 22]
23
[2, 14]
24
[12, 17]
25
[3, 13]
26
[0, 1]
27
[24, 16]
28
[15, 16]
29
[10, 2]
30
[14, 17]
31
[21]
32
[6, 8]
33
[13, 4]
34
[8, 17]
35
[19, 22]
36
[12, 0]
37
[10, 21]
38
[14, 17]
39
[23, 13]
40
[8, 10]
41
[17, 2]
42
[17, 18]
43
[3, 13]
45
[4, 0]
46
[11, 13]
47
[12, 17]
48
[12, 17]
49
[24, 0]
50
[6, 0]
51
[18, 17]
52
[4, 10]
53
[11, 4]
54
[5, 0]
55
[25, 12]
56
[24, 22]
57
[21, 23]
58
[2, 15]
59
[1, 3]
60
[1, 23]
61
[7, 5]
62
[13, 3]
63
[7, 18]
64
[5, 0]
65
[3, 0]
66
[11, 0]
68
[10, 4]
69
[15, 26]
70
[31, 13]
71
[39, 4]
72
[1, 34]
73
[26, 39]
74
[35, 25]
75
[22, 34]
76
[2, 4]
77
[19, 0]
78
[25, 14]
79
[15, 38]
80
[17, 18]
81
[7, 32]
82
[13, 30]
85
[31, 16]
86
[6, 37]
87
[16, 18]
88
[29, 0]
89
[8, 10]
90
[33, 20]
91
[20, 22]
92
[20, 17]
93
[16, 24]
94
[10, 12]
95


[8, 16]
784
[7, 18]
785
[14, 2]
786
[23, 9]
787
[17, 5]
788
[11, 12]
789
[5, 14]
791
[2, 11]
792
[22, 1]
793
[10, 15]
794
[19, 4]
795
[11, 20]
796
[6, 35]
797
[29, 28]
798
[31, 30]
799
[28, 2]
800
[8, 34]
801
[29, 28]
803
[43, 3]
804
[25, 42]
805
[22, 3]
806
[48, 24]
807
[9, 8]
808
[34, 20]
809
[22, 47]
810
[2, 28]
811
[32, 11]
812
[21, 28]
813
[6, 7]
814
[14, 3]
815
[21, 26]
816
[16, 3]
817
[3, 46]
818
[25, 24]
819
[18, 31]
820
[13, 24]
822
[24, 37]
824
[2, 3]
825
[20, 22]
826
[51, 3]
827
[8, 9]
828
[7, 6]
829
[33, 32]
830
[39, 34]
831
[16, 23]
832
[42, 34]
833
[28, 29]
834
[16, 38]
835
[39, 34]
836
[37, 26]
837
[31, 36]
838
[34, 19]
839
[23, 39]
840
[13, 50]
841
[48, 23]
843
[21, 3]
844
[30, 31]
845
[6, 3]
846
[39, 34]
847
[26, 28]
848
[27, 2]
849
[10, 30]
850
[34, 47]
851
[4, 40]
852
[33, 28]
853
[2, 30]
854
[48, 0]
855
[9, 8]
857
[8, 7]
858
[5, 13]
859
[37, 27]
860
[26, 28]
861
[1, 3]
862
[12, 34]
863
[9, 8]
864
[32, 13]
865
[21, 45]
866
[5, 16]
869
[28, 3]
871
[29, 2]
872
[11, 24]

[58, 2]
1557
[18, 21]
1558
[10, 55]
1559
[2, 58]
1560
[8, 40]
1561
[60, 2]
1562
[44, 43]
1563
[11, 39]
1564
[48, 50]
1565
[25, 5]
1566
[6, 29]
1567
[42, 41]
1568
[57, 11]
1569
[21, 34]
1570
[30, 29]
1571
[9, 43]
1575
[6, 4]
1576
[25, 11]
1577
[30, 45]
1578
[55, 41]
1579
[49, 28]
1580
[62, 53]
1581
[44, 16]
1583
[55, 54]
1584
[54, 25]
1585
[47, 44]
1586
[13, 23]
1587
[28, 22]
1588
[24, 44]
1589
[3, 10]
1590
[45, 2]
1593
[7, 8]
1594
[34, 18]
1595
[27, 1]
1597
[46, 45]
1598
[53, 46]
1599
[29, 30]
1600
[27, 28]
1601
[43, 4]
1602
[16, 23]
1603
[53, 54]
1604
[46, 45]
1605
[17, 29]
1606
[43, 58]
1607
[7, 8]
1608
[50, 58]
1609
[31, 43]
1610
[10, 18]
1612
[57, 13]
1613
[28, 27]
1615
[7, 8]
1616
[55, 56]
1617
[10, 3]
1618
[33, 18]
1619
[51, 48]
1620
[24, 22]
1621
[21, 23]
1622
[25, 32]
1623
[31, 28]
1624
[59, 60]
1625
[64, 52]
1626
[22, 8]
1627
[21, 41]
1628
[60, 59]
1630
[44, 43]
1631
[21, 32]
1632
[47, 60]
1634
[52, 42]
1636
[17, 15]
1637
[59, 60]
1638
[31, 11]
1639
[13, 23]
1640
[7, 15]
1641


2318
[6, 7]
2319
[21, 20]
2320
[2, 5]
2322
[13, 10]
2323
[5, 18]
2324
[21, 20]
2325
[9]
2326
[13, 6]
2327
[2, 15]
2328
[13, 5]
2329
[10, 11]
2330
[4, 2]
2331
[0, 6]
2332
[10, 11]
2333
[5, 8]
2334
[16, 7]
2335
[17, 18]
2336
[1, 0]
2337
[11, 0]
2338
[0, 11]
2339
[17, 18]
2340
[20, 9]
2341
[17, 18]
2342
[2, 5]
2343
[21, 5]
2344
[6, 7]
2345
[22, 8]
2346
[13, 8]
2348
[8, 13]
2349
[0, 1]
2350
[15, 7]
2351
[20]
2352
[21, 20]
2353
[1, 3]
2354
[4, 5]
2355
[16, 14]
2357
[12, 8]
2358
[17, 18]
2360
[15, 7]
2361
[7, 16]
2363
[60, 59]
2364
[69, 40]
2365
[39, 17]
2366
[53, 0]
2367
[65, 41]
2368
[40, 28]
2369
[30, 29]
2370
[41, 43]
2371
[12, 53]
2372
[10, 53]
2374
[29, 62]
2375
[49, 24]
2376
[11, 14]
2380
[58, 16]
2381
[72, 41]
2382
[57, 40]
2384
[5, 6]
2386
[40, 42]
2387
[5, 6]
2388
[3, 18]
2389
[23, 37]
2390
[66, 63]
2392
[13, 28]
2393
[54, 58]
2394
[67, 11]
2395
[12, 9]
2397
[23, 18]
2399
[75, 19]
2400
[0, 3]
2401
[49, 32]
2402
[41, 16]
2403
[10, 24]
2404
[55, 27]
2405
[13, 18]
2406
[69, 58]
2407
[

[16, 17]
3092
[17, 16]
3093
[17, 21]
3094
[24, 27]
3095
[2, 0]
3096
0.6922465208747515
Done


In [540]:
count  = 0
for index,row in df_result_devel.iterrows():
    sentence = row['sentence']
    predicted_sentence = row['predicted_sentence']
    if(sentence==predicted_sentence):
        count = count + 1
        df_result_devel.at[index, 'sentence_found'] = True
print(count*1.0/len(df_result_devel))

0.6922465208747515


In [113]:
def get_number_of_common_entities(ans,question):
    question_NE = set(x[1] for x in question)
    answer_NE = set(x[1] for x in ans)
    sum_keywords=0
    for qkey in question_NE:
        if qkey in answer_NE:
            sum_keywords+=1
#     print(ans,question)
    return sum_keywords

In [541]:
df_result_devel = df_result_devel[df_result_devel['sentence_found']==True]
print(len(df_result_devel))

1741


In [267]:
count = 0

df_result_devel1['predicted_label'] = None

for index,row in df_result_devel1.iterrows():
    sentence = row['predicted_sentence']
    question = row['question']
    question_NER = row['NE_question']
    sentence_id = row['predicted_sentence_id']
    paragraph_id = row['predicted_paragraph']
    docid = row['id']
    answer = row['answer']
    sentence_NER = NER_corpus[docid][paragraph_id][sentence_id]
    sentence_keywords = get_keyword(sentence)
    question_keywords=get_keyword(question)
    predicted_label = model.predict(vectorizer.transform(
        get_feature_question(question,answer_keywords,get_question_type(question))))[0]
    
    df_result_devel1.at[index, 'predicted_label'] = predicted_label
    
    answer_type = None
    predicted_answer = None
    for NE in sentence_NER:
        if (label[NE[1]]==predicted_label):
            answer_type = NE[1]
            predicted_answer = NE[0]
            break
#     print(index,answer,predicted_label,answer_type,predicted_answer)
    if(answer_type):
        df_result_devel1.at[index, 'predicted_answer'] = predicted_answer
        count = count + 1
#     print(index)
print(count)
#     else:
#         print(sentence,""predicted_sentence)
#         print(index,answer,predicted_label,answer_type)
#         break
#     print(index,question,paragraph_id,sentence_id,
#           sentence,sentence_NER,sentence_keywords,question_keywords,answer,predicted_label)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


45


In [270]:
len(df_result_devel)

72

In [278]:
count = 0
for index,row in df_result_devel1.iterrows():
    ans = row['label']
    pred = row['predicted_label']
    if(ans == pred):
        count  = count + 1
print(count)

32


In [280]:
import difflib

count = 0
count1 = 0
count3 = 0
for index,row in df_result_devel1.iterrows():
    ans = row['answer']
    pred = row['predicted_answer']
    if(row['label']==row['predicted_label']):
        if(ans == pred):
            count = count + 1
        else:
            if(pred):
                seq=difflib.SequenceMatcher(None, ans,pred)
                d=seq.ratio()*100
                if (d>20.00):
    #                 print(index,row['question'])
                    print(row['label'],"~~",row['predicted_label'],'~~',ans,"~~",pred)
                    count1 = count1 + 1
                else:
    #                 print(index,row['question'])
                    print(row['label'],"~~",row['predicted_label'],'~~',ans,"~~",pred)
                    count3 = count3 + 1
            else:
                print("strange",pred,row['answer'],"~~",row["predicted_answer"])
print(count,count1,count3)

NUMBER ~~ NUMBER ~~ second largest ~~ second
NUMBER ~~ NUMBER ~~ 4,100 gallons ~~ six
TIME ~~ TIME ~~ '07 ~~ november 2010
PERSON ~~ PERSON ~~ blue gene ~~ gene
PERSON ~~ PERSON ~~ albert l. williams ~~ thomas j. watson
NUMBER ~~ NUMBER ~~ 34 ~~ first
TIME ~~ TIME ~~ 2013 ~~ 2004
LOCATION ~~ LOCATION ~~ the u.s. south ~~ u.s.
strange None charles ranlett flint ~~ None
NUMBER ~~ NUMBER ~~ 78,000 gallons ~~ 78,000
NUMBER ~~ NUMBER ~~ five nobel prizes ~~ five
NUMBER ~~ NUMBER ~~ fourth largest ~~ second
NUMBER ~~ NUMBER ~~ 26 million ~~ first
NUMBER ~~ NUMBER ~~ 5.5 ~~ 5.5 million
NUMBER ~~ NUMBER ~~ 53 ~~ 400
LOCATION ~~ LOCATION ~~ tennessee ~~ oak ridge
16 8 7


In [ ]:
df_result_devel=pd.DataFrame(columns=['id','question','paragraph','retrieved paras','predicted_paragraph',
                                      'paragraph_found','sentence','predicted_sentence','predicted_sentence_id',
                                      'sentence_found','answer','predicted_answer','NE_question','label'])
df_devel1=df_devel.iloc[0:100]
count = 0
for index, row in df_devel1.iterrows():
    question=row['question']
    docid=row['docid']
    ida=index
    question_keywords=get_keyword(question)
    results = query_vsm(question_keywords, vsm_inverted_index_all[docid])
    documents_ranked=results.most_common(10) 
    best_dic = set()
    cc = 0
    found = False
    possible_par=[par[0] for par in documents_ranked]
    best_paragraph_id = -1
    par_retrieved = False
    for doc in documents_ranked:
        if(cc>1):
            break
        cc = cc + 1
        best_dic.add(doc[0])
    if(row['answer_paragraph'] in best_dic):
        count = count + 1
        par_retrieved = True
        best_paragraph_id = row['answer_paragraph']
    df_result_devel.loc[ida] = [docid,question,row['answer_paragraph'],
                                possible_par,best_paragraph_id,
                                par_retrieved,row['answer_found'],None,None,False,row['text'],None,
                                row['NE_question'],row['label']]

print(count*1.0/len(df_devel1))

In [490]:
len(df_testing)

3618

In [ ]:
df_result_test=pd.DataFrame(columns=['id','question','paragraph','retrieved paras','predicted_paragraph',
                                      'paragraph_found','sentence','predicted_sentence','predicted_sentence_id',
                                      'sentence_found','answer','predicted_answer','NE_question','label','predicted_paragraph_final'])
df_testing1=df_testing.iloc[0:10]
count = 0
for index, row in df_testing.iterrows():
    question=row['question']
    docid=row['docid']
    ida=index
    question_keywords=get_keyword(question)
    results = query_vsm(question_keywords, vsm_inverted_index_all[docid])
    documents_ranked=results.most_common(2)
    possible_par=[par[0] for par in documents_ranked]
    df_result_test.loc[ida] = [docid,question,None,
                                possible_par,possible_par,
                                None,None,None,None,False,None,None,
                                row['NE_question'],None,None]
    print(index)

In [492]:
df_result_test

,id,question,paragraph,retrieved paras,predicted_paragraph,paragraph_found,sentence,predicted_sentence,predicted_sentence_id,sentence_found,answer,predicted_answer,NE_question,label,predicted_paragraph_final
0,410,Modern browser support standards-based and def...,None,"[22, 17]","[22, 17]",None,None,Modern web browsers support a combination of s...,2,False,None,None,"[(modern, MISC)]",None,22
1,410,What do people typically call a web browser?,None,"[2, 20]","[2, 20]",None,None,"In addition, all major web browsers have some ...",2,False,None,None,[],None,20
2,410,What is it called when content is changed from...,None,"[15, 20]","[15, 20]",None,None,"HTML and associated content (image files, form...",1,False,None,None,[],None,15
3,410,What platform is a browser used on?,None,"[1, 9]","[1, 9]",None,None,Although browsers are primarily intended to us...,0,False,None,None,[],None,1
4,410,When was Firefox released?,None,"[6, 12]","[6, 12]",None,None,That browser would eventually evolve into Fire...,1,False,None,None,"[(firefox, ORGANIZATION)]",None,6
5,410,Who released the Internet Explorer browser?,None,"[4, 8]","[4, 8]",None,None,"Bundled with Windows, Internet Explorer gained...",1,False,None,None,"[(internet, MISC), (explorer, TITLE)]",None,4
6,410,When was the first browser created?,None,"[7, 3]","[7, 3]",None,None,"In 1993, browser software was further innovate...",0,False,None,None,"[(first, ORDINAL)]",None,3
7,410,HTTP Secure is supported by what?,None,"[21, 14]","[21, 14]",None,None,Most browsers support HTTP Secure and offer qu...,0,False,None,None,[],None,21
8,410,Who released Mosaic?,None,"[3, 4]","[3, 4]",None,None,"In 1993, browser software was further innovate...",0,False,None,None,[],None,3
9,410,Who invented the first browser?,None,"[2, 7]","[2, 7]",None,None,The first web browser was invented in 1990 by ...,0,False,None,None,"[(first, ORDINAL)]",None,2


In [494]:
count  = 0
for index,row in df_result_test.iterrows():
    NE_question = row['NE_question']
    question = row['question']
    docid = row['id']
    candidate_answers = {}
    possible_paragraphs = row['predicted_paragraph']
    sentences_check = Counter()
    if(len(possible_paragraphs)>0):
        for paragraph_id in possible_paragraphs:
            paragraph = df_docs.iloc[row['id']]['text'][paragraph_id]
            candidate_answers[paragraph_id] = tokenizer.tokenize(paragraph)
            question_keywords=get_keyword(question)
            for sentence_index,sentence in enumerate(candidate_answers[paragraph_id]):
                NER_sentence=NER_corpus[docid][paragraph_id][sentence_index]
                answer_sentence_keywords = get_keyword(sentence)
                common_keywords=get_number_of_common_kewyords(question_keywords,answer_sentence_keywords)
                common_entities = get_number_of_common_entities(NER_sentence,NE_question)

                # longest exact sequence of keywords
                longest_exact_sequence=0
                for i in range(len(question_keywords)):
                    if i < len(answer_sentence_keywords):
                        if question_keywords[i] in answer_sentence_keywords[i]:
                            longest_exact_sequence+=1

                # proximity
                proximity=0
                question_keywords_span=question_keywords.copy()
                index_qk=0
                while len(question_keywords_span)>0 and index_qk<len(question_keywords_span):
                    proximity+=1
                    if question_keywords_span[index_qk] in answer_sentence_keywords:
                        question_keywords_span.pop(index_qk)
                    index_qk+=1

                # n-gram overlap
                bigrams_question =  nltk.bigrams([lemmatize(token) for token in nltk.word_tokenize(question)])
                ngram_overlap=0

                for bigram_question in bigrams_question:
                    bigrams_sentence = nltk.bigrams([lemmatize(token) for token in nltk.word_tokenize(sentence)])
                    for bigram_sentence in bigrams_sentence:
                        if bigram_question == bigram_sentence:
                            ngram_overlap+=1

                sentence_score = common_keywords + common_entities + longest_exact_sequence - proximity + ngram_overlap
                df_best_sentences.loc[len(df_best_sentences)]=[row['id'],
                                                           paragraph_id,row['paragraph'],sentence_index,
                                                           sentence,sentence_score,row['sentence']]
                sentences_check[(paragraph_id,sentence_index)] = sentence_score
    #             print(paragraph_id,sentence_index,sentence_score)

        best = sentences_check.most_common()
    #     print(sentences_check)
        df_result_test.at[index, 'predicted_paragraph_final'] = best[0][0][0]
        df_result_test.at[index, 'predicted_sentence_id'] = best[0][0][1]
        df_result_test.at[index, 'predicted_sentence'] = candidate_answers[best[0][0][0]][best[0][0][1]]
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059


3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617


In [495]:
df_result_test.head()

,id,question,paragraph,retrieved paras,predicted_paragraph,paragraph_found,sentence,predicted_sentence,predicted_sentence_id,sentence_found,answer,predicted_answer,NE_question,label,predicted_paragraph_final
0,410,Modern browser support standards-based and def...,None,"[22, 17]","[22, 17]",None,None,Modern web browsers support a combination of s...,2,False,None,None,"[(modern, MISC)]",None,22
1,410,What do people typically call a web browser?,None,"[2, 20]","[2, 20]",None,None,"In addition, all major web browsers have some ...",2,False,None,None,[],None,20
2,410,What is it called when content is changed from...,None,"[15, 20]","[15, 20]",None,None,"HTML and associated content (image files, form...",1,False,None,None,[],None,15
3,410,What platform is a browser used on?,None,"[1, 9]","[1, 9]",None,None,Although browsers are primarily intended to us...,0,False,None,None,[],None,1
4,410,When was Firefox released?,None,"[6, 12]","[6, 12]",None,None,That browser would eventually evolve into Fire...,1,False,None,None,"[(firefox, ORGANIZATION)]",None,6


In [523]:
df_result_test.to_pickle("df_result_test1.pkl")
print ("done")

done


In [543]:
df_result_devel.to_pickle("df_result_devel.pkl")
print ("done")

done


In [498]:
df_result_test1=df_result_test.iloc[0:10]

In [534]:
df_result=pd.DataFrame(columns=['id','answer'])
df_result_test['predicted_label'] = None
for index,row in df_result_test.iterrows():
    sentence = row['predicted_sentence']
    question = row['question']
    question_NER = row['NE_question']
    sentence_id = row['predicted_sentence_id']
    paragraph_id = row['predicted_paragraph_final']
    docid = row['id']
    if(paragraph_id):
        sentence_NER = NER_corpus[docid][paragraph_id][sentence_id]
        sentence_keywords = get_keyword(sentence)
        question_keywords=get_keyword(question)
        predicted_label = model.predict(vectorizer.transform(
            get_feature_question(question,answer_keywords,get_question_type(question))))[0]

        df_result_test.at[index, 'predicted_label'] = predicted_label

        answer_type = None
        predicted_answer = None
        list_match_answer_type=[]
        list_match_pred_answer=[]
        first_match=True
        for NE in sentence_NER:
            if (label[NE[1]]==predicted_label):            
                answer_type = NE[1]
                predicted_answer = NE[0]
                list_match_answer_type.append(answer_type)
                list_match_pred_answer.append(predicted_answer)
                if first_match:
                    print(index,"NER best",list_match_pred_answer)
                    break
            else:
                first_match=False
#         print(len(list_match_pred_answer))
        if (len(list_match_pred_answer)==0):
            for NE in sentence_NER:            
                answer_type = NE[1]
                predicted_answer = NE[0]
                if(predicted_answer.lower() not in [x.lower() for x in list_match_pred_answer]):
                    list_match_answer_type.append(answer_type)
                    list_match_pred_answer.append(predicted_answer)
            print(index,"NER",list_match_pred_answer)
        if (len(list_match_pred_answer)==0):
            for word,pos in nltk.pos_tag(nltk.word_tokenize(sentence)):
                if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'CD'):
                    if(word.lower() not in [x.lower() for x in list_match_pred_answer]):
                        list_match_pred_answer.append(word)
                        list_match_answer_type.append(pos)
            print(index,"POS",list_match_pred_answer)
        if (len(list_match_pred_answer)==0):
            print(index, "keyword")
            for word in sentence:
                if word not in stopwords:
                    if(word.lower() not in [x.lower() for x in list_match_pred_answer]):
                        list_match_pred_answer.append(word)

        df_result_test.at[index, 'predicted_answer'] = " ".join(list_match_pred_answer)
        df_result.loc[len(df_result)]=[index," ".join(list_match_pred_answer)]
    else:
        df_result.loc[len(df_result)]=[index,""]

0 NER []
0 POS ['Modern', 'web', 'browsers', 'combination', 'HTML', 'XHTML', 'way']
1 NER []
1 POS ['addition', 'web', 'browsers', 'form', 'aggregator']
2 NER ['css']
3 NER ['world wide web']
6 NER best ['1993']
7 NER []
7 POS ['browsers', 'HTTP', 'Secure', 'ways', 'web', 'cache', 'download', 'history', 'form', 'search', 'cookies']
12 NER []
12 POS ['Browsers', 'components', 'e-mail', 'Usenet', 'news', 'Internet', 'Relay', 'Chat', 'IRC', 'suites']
14 NER []
14 POS ['browser', 'extension', 'computer', 'program', 'functionality', 'web']
16 NER best ['january 2009']
17 NER []
17 POS ['browsers', 'variety', 'prefixes', 'https', 'ftp', 'File', 'Transfer', 'Protocol', 'files']
18 NER ['today']
20 NER []
20 POS ['web', 'browsers', 'features', 'user', 'interfaces', 'support', 'HTML', 'variety', 'file', 'formats', 'protocols']
22 NER best ['1996']
24 NER ['year', '8 %', '16 %', 'august 2011']
25 NER []
25 POS ['addition', 'web', 'browsers', 'form', 'aggregator']
26 NER best ['december 2011']
27

276 NER ['india', 'pro-tibet', 'new delhi', 'april 17', '2.3', '1.5', '70']
277 NER ["people 's daily", 'chinese', 'orderly']
278 NER best ['turkey']
279 NER ['macau', 'april 26', 'cyberctm.com']
281 NER ['macao daily news', 'macanese']
282 NER ['june 2008', "beijing games ' organizing committee", 'paralympic games']
285 NER best ['lê minh phiếu']
287 NER ['bangkok', 'chinese embassy']
288 NER ['eiffel', 'tower', 'notre dame']
289 NER ['french', 'libération', 'chinese state', 'xinhua', 'olympic']
290 NER best ['april 16']
291 NER ['libération', 'chinese']
292 NER best ['mayor']
293 NER ['tibetan']
295 NER ['tibetan']
296 NER ['april 20', "people 's daily", 'chinese', 'olympics']
297 NER ['third', 'paris', 'jin jing', 'pro-tibet', 'independent']
298 NER ['april 7 , 2008', 'two days', 'three', 'tibetan', 'golden gate', 'bridge', 'two', 'one']
300 NER ['japan', 'nagano', 'the 1998 winter', 'olympics', 'april 26']
301 NER ['chinese', 'tibetan', 'city hall', 'green party']
302 NER ['french'

573 NER ['today']
575 NER best ['the next few decades']
578 NER ['hampshire county cricket club', 'rose bowl', 'west end', 'previously', 'antelope ground']
579 NER ['2015']
581 NER ['hampshire constabulary', 'southampton', 'currently', 'year', 'the last three years']
582 NER ['southampton', 'southampton football club', 'premier league', "st mary 's stadium", '2001', '103-year-old']
586 NER ['southampton', 'two']
587 NER best ['24 february 1964']
588 NER best ['1904']
589 NER best ['duchess']
591 NER best ['charter']
592 NER best ['three']
594 NER best ['king']
598 NER []
598 POS ['controversy', 'crime', 'statisitics', 'inconsistencies', 'police', 'forces', 'methodologies']
600 NER best ['the 1760s']
604 NER best ['500,000']
605 NER best ['the centuries']
606 NER ['ocean village', '# 100m']
607 NER best ['southampton']
609 NER best ['11']
610 NER ['millais gallery', 'southampton solent university', 'john hansard gallery', 'southampton university', 'art house', 'above bar street']
611 NE

911 POS ['science', 'mixture', 'icy', 'fluid']
912 NER ['sun', 'neptune', 'one', '55', '218']
913 NER ['neptune', 'roman']
914 NER ['earth', '2', '1989', 'night']
916 NER []
916 POS ['candidates', 'waves', 'interior', 'dissipate', 'atmosphere']
917 NER ['neptune', 'uranus']
918 NER best ['kant']
919 NER best ['hobbes']
921 NER []
921 POS ['culture', 'refers', 'ideas', 'individuals', 'values', 'belief', 'system', 'rules', 'norms', 'morals', 'language', 'organizations', 'institutions']
922 NER best ['raimon panikkar']
926 NER ['two']
927 NER ['romantic', 'germany', 'nationalist', 'austro-hungarian empire']
928 NER []
928 POS ['example', 'forests', 'end', 'ice', 'age', 'plants', 'domestication', 'invention', 'agriculture', 'turn', 'innovations', 'shifts', 'dynamics']
929 NER []
929 POS ['perspectives', 'culture', 'distinctions', 'elite', 'classes', 'access', 'capital']
930 NER best ['matthew arnold']
932 NER best ['tylor']
934 NER []
934 POS ['aspect', 'culture', 'concept', 'anthropology'

1172 NER ['roman', 'north africa', 'carthage', 'bardo museum', 'tunis', 'tunisia']
1173 NER ['the mid-16th century', 'taungoo']
1176 NER ['yangon']
1177 NER ['myanmar', 'burma']
1178 NER ['one', 'irrawaddy', 'burmese', '16', 'three']
1180 NER ['myanmar', 'burma']
1181 NER best ['konbaung']
1182 NER ['previously', 'myanmar', '7', '2012', '6']
1183 NER best ['october 2012']
1184 NER ['once', 'anglo-burmese', '1958', 'australia', 'uk']
1186 NER ['shan states']
1187 NER ['myanmar', 'burma']
1189 NER ['human rights watch', 'burmese']
1190 NER best ['general']
1191 NER best ['10 %']
1192 NER ['the 16th century', 'taungoo dynasty', 'southeast', 'asia']
1193 NER ['myanmar', 'burma']
1196 NER ['myanmar', 'burma']
1197 NER ['myanmar', 'burma']
1198 NER ['burma', 'chinese', '1770', 'lan na', '1776']
1199 NER ['myanmar', 'burma']
1200 NER ['burma', 'united kingdom']
1201 NER ['myanmar', 'burma']
1202 NER ['burmese']
1203 NER ['burmese', 'rohingya muslims', 'violence']
1204 NER ['two', 'myanmar', '

...

2709 NER ['tuvalu media department', 'tuvalu', 'funafuti']
2710 NER ['tuvalu', 'australia', 'new zealand', 'kioa', 'fiji', '2006', 'maatia toafa', 'prime minister', '2004']
2711 NER ['tuvalu']
2712 NER best ['doug munro']
2713 NER best ['the past']
2715 NER best ['26']
2716 NER best ['charles hedley']
2718 NER ['janet nicoll', 'three', 'ellice islands', 'fanny', 'funafuti', 'niutao', 'nanumea', 'jane resture']
2719 NER ['1974', 'british', 'tuvalu', 'assembly', 'fale']
2722 NER ['tuvalu', '2002']
2723 NER ['tuvalu', 'the mid-19th century']
2724 NER ['current', 'future']
2726 NER ['tuvalu', 'the mid-19th century']
2727 NER ['tuvalu', 'pacific community', 'pacific islands forum', 'united nations']
2728 NER ['tuvalu']
2729 NER ['sixth', 'augmented foundation programme', 'tuvalu']
2730 NER best ['year']
2731 NER best ['1892']
2732 NER ['majuro declaration', '5 september 2013', 'tuvalu', '100 %', '2013 and 2020', '95 %', '5 %']
2733 NER best ['edgar waite']
2735 NER best ['2014']
2736 NER []

2974 POS ['disk', 'drive', 'software', 'CDs', 'media', 'third-party', 'hardware']
2977 NER best ['september 1986']
2978 NER best ['1994']
2979 NER []
2979 POS ['disk', 'drive', 'software', 'CDs', 'media', 'third-party', 'hardware']
2980 NER best ['april 1984']
2981 NER best ['august 1987']
2983 NER best ['ten']
2984 NER best ['2002']
2987 NER best ['1998']
2988 NER best ['2011']
2989 NER best ['1987']
2991 NER best ['virbhadra singh']
2993 NER best ['governor']
2994 NER best ['himachal']
2995 NER ['himachal', 'khatri', 'hindu', 'sikh brahmin caste bhatra', 'rajputs', 'gujjars', 'gaddis', 'ghirth', 'rathis', 'kolis', 'sood', 'kinnars', 'sulehria', 'himachal pradesh']
2996 NER ['kangra', 'dharamshala']
2997 NER ['100 %']
2998 NER ['18th', 'indian', 'union']
2999 NER []
2999 POS ['Hima', 'Sanskrit', 'meaning', 'state', 'name', 'lap', 'Himalayas']
3000 NER ['himachal pradesh']
3003 NER []
3003 POS ['hill', 'stations', 'state', 'places', 'country']
3004 NER ['all india radio']
3006 NER ['ka

3208 POS ['Magistrates', 'opinion', 'acts', 'augur', 'divine', 'observations', 'templum', 'act', 'sacrifice']
3209 NER ['roman']
3210 NER ['hellenization of latin', 'roman', 'greek', 'olympians', 'two']
3211 NER ['christian', 'roman']
3212 NER ['ovid']
3213 NER best ['atheism']
3215 NER ['vestal']
3216 NER ['81 bc', 'murder']
3217 NER ['roman', 'rome']
3218 NER []
3218 POS ['Archaeology', 'use', 'spells', 'defixiones', 'papyri', 'voodoo', 'dolls', 'era']
3219 NER ['cult', 'vesta']
3220 NER []
3220 POS ['cults', 'individuals', 'benefit', 'communities']
3221 NER ['st augustine', 'st ambrose', 'christian']
3222 NER ['christian']
3224 NER best ['varro']
3225 NER ['roman']
3226 NER ['graeco-roman', 'persian']
3227 NER best ['greek']
3228 NER best ['bona dea']
3229 NER best ['pliny the elder']
3230 NER ['fire', 'vesta']
3231 NER best ['constantine']
3232 NER []
3232 POS ['Prodigies', 'transgressions', 'order', 'cosmos', '–', 'signs', 'divine', 'anger', 'conflict', 'misfortune']
3233 NER []
3

3389 NER ['chevron']
3392 NER ['congress']
3393 NER []
3393 POS ['agencies', 'regulations']
3394 NER best ['decades']
3395 NER ['1996', 'moldovan', 'romanian', 'president', 'mircea snegur', '2003', 'two']
3396 NER best ['beijing']
3397 NER ['italian', 'latin-derived florentine tuscan']
3398 NER []
3398 POS ['criterion', 'dialects', 'languages', 'authority', 'notion']
3399 NER []
3399 POS ['dialect', 'class', 'sociolect', 'group', 'ethnolect', 'regiolect']
3400 NER ['sardinian', 'italian', 'italo-dalmatian', 'campidanese sardinian', 'logudorese sardinian']
3402 NER ['italian', 'republicanism', 'nationalism', 'alessandro manzoni']
3403 NER ['one', 'swedish', 'interlingua', 'spanish', 'portuguese', 'italian']
3404 NER []
3404 POS ['dialect', 'class', 'sociolect', 'group', 'ethnolect', 'regiolect']
3405 NER best ['tunisia']
3406 NER best ['1996']
3407 NER ['italian', 'spanish', 'french']
3408 NER []
3408 POS ['cities', 'dialects', 'countryside']
3410 NER ['swiss german', 'german']
3412 NER

3616 NER ['one', 'europe', 'turkey', 'india', '18th', 'early 19th century', 'france']
3617 NER ['one', 'europe', 'turkey', 'india', '18th', 'early 19th century', 'france']


In [536]:
df_result.to_csv('prediction/output.csv',index=False)